<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MYEMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple energy-based model (EBM)
class SimpleEBM(nn.Module):
    def __init__(self, input_size):
        super(SimpleEBM, self).__init__()
        self.linear1 = nn.Linear(input_size, 128)
        self.linear2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        energy = self.linear2(x)
        return energy

# Generate some sample data
input_size = 10
data = torch.randn(100, input_size)

# Initialize the EBM and optimizer
model = SimpleEBM(input_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for batch in data:
        optimizer.zero_grad()
        energy = model(batch)
        loss = torch.mean(energy)  # Minimize the energy
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Example usage: Calculate energy for a new sample
new_sample = torch.randn(1, input_size)
energy = model(new_sample)
print(f"Energy for new sample: {energy.item():.4f}")

Epoch [1/100], Loss: -2.0944
Epoch [2/100], Loss: -5.1736
Epoch [3/100], Loss: -10.2510
Epoch [4/100], Loss: -17.8607
Epoch [5/100], Loss: -28.2417
Epoch [6/100], Loss: -41.5464
Epoch [7/100], Loss: -57.8382
Epoch [8/100], Loss: -77.1841
Epoch [9/100], Loss: -99.4283
Epoch [10/100], Loss: -124.7059
Epoch [11/100], Loss: -153.1870
Epoch [12/100], Loss: -185.0000
Epoch [13/100], Loss: -220.0206
Epoch [14/100], Loss: -258.3177
Epoch [15/100], Loss: -299.8869
Epoch [16/100], Loss: -344.5794
Epoch [17/100], Loss: -392.2733
Epoch [18/100], Loss: -443.0975
Epoch [19/100], Loss: -497.2932
Epoch [20/100], Loss: -554.3760
Epoch [21/100], Loss: -614.5308
Epoch [22/100], Loss: -677.6592
Epoch [23/100], Loss: -743.7884
Epoch [24/100], Loss: -813.0590
Epoch [25/100], Loss: -885.2091
Epoch [26/100], Loss: -960.2931
Epoch [27/100], Loss: -1038.0800
Epoch [28/100], Loss: -1118.5536
Epoch [29/100], Loss: -1201.6860
Epoch [30/100], Loss: -1287.4641
Epoch [31/100], Loss: -1375.9611
Epoch [32/100], Loss: -

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Load MNIST dataset
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transforms.ToTensor()
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Load MNIST test dataset
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the ImageEBM with mixed activations
class ImageEBM(nn.Module):
    def __init__(self):
        super(ImageEBM, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 28 * 28, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))  # ReLU after convolution
        x = x.view(-1, 8 * 28 * 28)
        x = torch.sigmoid(self.fc1(x))  # Sigmoid in fully connected layer
        energy = self.fc2(x)
        return energy

# Initialize the EBM, optimizer, and learning rate scheduler
model = ImageEBM()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop with reduced energy regularization
num_epochs = 5
alpha = 0.1  # Reduced regularization strength
for epoch in range(num_epochs):
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        energy = model(data)

        # Add energy regularization to the loss
        loss = torch.mean(energy) + alpha * torch.mean(torch.relu(-energy))

        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}"
            )

    scheduler.step()  # Update learning rate

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Epoch [1/5], Batch [1/938], Loss: 0.1992
Epoch [1/5], Batch [101/938], Loss: -0.7339
Epoch [1/5], Batch [201/938], Loss: -0.7856
Epoch [1/5], Batch [301/938], Loss: -0.8311
Epoch [1/5], Batch [401/938], Loss: -0.8778
Epoch [1/5], Batch [501/938], Loss: -0.9235
Epoch [1/5], Batch [601/938], Loss: -0.9693
Epoch [1/5], Batch [701/938], Loss: -1.0145
Epoch [1/5], Batch [801/938], Loss: -1.0597
Epoch [1/5], Batch [901/938], Loss: -1.1048
Epoch [2/5], Batch [1/938], Loss: -1.1221
Epoch [2/5], Batch [101/938], Loss: -1.1673
Epoch [2/5], Batch [201/938], Loss: -1.2124
Epoch [2/5], Batch [301/938], Loss: -1.2575
Epoch [2/5], Batch [401/938], Loss: -1.3025
Epoch [2/5], Batch [501/938], Loss: -1.3477
Epoch [2/5], Batch [601/938], Loss: -1.3928
Epoch [2/5], Batch [701/938], Loss: -1.4377
Epoch [2/5], Batch [801/938], Loss: -1.4829
Epoch [2/5], Batch [901/938], Loss: -1.5279
Epoch [3/5], Batch [1/938], Loss: -1.5451
Epoch [3/5], Batch [101/938], Loss: -1.5901
Epoch [3/5], Batch [201/938], Loss: -1.

In [10]:
# Load MNIST test dataset
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    print(f"Eval Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Eval Loss: -3.1242


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the ImageEBM for CIFAR-10 (color images)
class ImageEBM(nn.Module):
    def __init__(self):
        super(ImageEBM, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Input channels = 3 for color
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Adjust input size for CIFAR-10
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 8 * 8)  # Adjust size for CIFAR-10
        x = torch.sigmoid(self.fc1(x))
        energy = self.fc2(x)
        return energy

# Initialize the EBM, optimizer, and learning rate scheduler
model = ImageEBM()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop with energy regularization
num_epochs = 5  # You might need more epochs for CIFAR-10
alpha = 0.1
for epoch in range(num_epochs):
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        energy = model(data)

        # Add energy regularization to the loss
        loss = torch.mean(energy) + alpha * torch.mean(torch.relu(-energy))

        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}"
            )

    scheduler.step()  # Update learning rate

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Batch [1/782], Loss: -0.3356
Epoch [1/5], Batch [101/782], Loss: -3.6440
Epoch [1/5], Batch [201/782], Loss: -4.3777
Epoch [1/5], Batch [301/782], Loss: -5.0892
Epoch [1/5], Batch [401/782], Loss: -5.8079
Epoch [1/5], Batch [501/782], Loss: -6.5292
Epoch [1/5], Batch [601/782], Loss: -7.2430
Epoch [1/5], Batch [701/782], Loss: -7.9535
Epoch [2/5], Batch [1/782], Loss: -8.5346
Epoch [2/5], Batch [101/782], Loss: -9.2422
Epoch [2/5], Batch [201/782], Loss: -9.9488
Epoch [2/5], Batch [301/782], Loss: -10.6550
Epoch [2/5], Batch [401/782], Loss: -11.3972
Epoch [2/5], Batch [501/782], Loss: -12.1229
Epoch [2/5], Batch [601/782], Loss: -12.8440
Epoch [2/5], Batch [701/782], Loss: -13.5626
Epoch [3/5], Batch [1/782], Loss: -14.1661
Epoch [3/5], Batch [101/782], Loss: -14.9152
Epoch [3/5], Batch [201/782], Loss: -15.6515
Epoch [3/5], Batch [301/782], Loss: -16.4029
Epoch [3/5], Batch [401/782], Loss: -17.1

## eval for CIFAR-10

In [14]:
import torch
from torchvision import datasets, transforms

# Load CIFAR-10 test dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    print(f"Eval Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Files already downloaded and verified
Eval Loss: -34.9541
